In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

app = FaceAnalysis(providers=['CUDAExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

In [ ]:
input_video = "/home/thesus/Apps/Assets/video_many_trim.mp4"

In [ ]:
import cv2

cap = cv2.VideoCapture(input_video)

ret, frame = cap.read()

if ret:
    faces = app.get(frame)
    rimg = app.draw_on(frame, faces)

    plt.imshow(rimg[:, :, ::-1])

In [ ]:
embedding = faces[0].normed_embedding

In [ ]:
from modules.face_resolver import FaceResolver

fr = FaceResolver(input_video, embedding)

bboxes = fr.get_bboxes()

In [ ]:
len(bboxes)

In [ ]:
import gradio as gr
import cv2

def process_video(video_file):
    video_capture = cv2.VideoCapture(video_file)
    frames = []
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        frames.append(frame)
    video_capture.release()
    return frames

def update_frame(frames, frame_index):
    return frames[frame_index]

def main():
    with gr.Blocks() as demo:
        video_file = gr.File(label="Upload Video")
        frames = gr.State([])
        frame_index = gr.Slider(minimum=0, maximum=1, step=1, label="Frame Index", interactive=True)
        frame_output = gr.Image(label="Frame Preview")

        def load_video(video_file):
            frames_list = process_video(video_file.name)
            return frames_list, gr.update(maximum=len(frames_list) - 1)

        video_file.change(load_video, inputs=[video_file], outputs=[frames, frame_index])
        frame_index.change(update_frame, inputs=[frames, frame_index], outputs=[frame_output])

    demo.launch()

if __name__ == "__main__":
    main()

In [ ]:
import gradio as gr
import cv2
import numpy as np
from insightface.app import FaceAnalysis
import os

# Assuming helpers.file_check as fc is correctly configured
import helpers.file_check as fc

class FaceResolver:
    def __init__(self, video_path):
        self.video_path = video_path
        self.face_detector = FaceAnalysis(providers=['CUDAExecutionProvider'])
        self.face_detector.prepare(ctx_id=0, det_size=(640, 640))
        
        self.selected_embedding = 0
        self.faces = []

        self.presence_mask = []
        self.bboxes = []
        self.kps = []

    def get_faces(self, frame):
        faces = self.face_detector.get(frame)
        self.faces = faces
    
    def get_embedding(self, index, faces):
        self.selected_embedding = faces[index].normed_embedding

    def _compare_embeddings(self, emb1, emb2, strength = 1.2):
        return np.linalg.norm(emb1 - emb2) < strength

    def get_preview(self, similarity_strength):
        bboxes = []
        presence_mask = []
        keypoints = []

        video = cv2.VideoCapture(self.video_path)

        # Metadata
        fps = video.get(cv2.CAP_PROP_FPS)
        frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Writer Object for output video
        preview_path = os.path.join(fc.MEDIA_DIR, 'preview.mp4')
        writer = cv2.VideoWriter(preview_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

        # Loop through the video
        while True:
            ret, frame = video.read()
            if not ret:
                break
            faces = self.face_detector.get(frame)
            if faces:
                present = False
                for face in faces:
                    bbox = face.bbox.astype(int)
                    landmark = face.kps.astype(int)
                    embedding = face.normed_embedding
                    if self._compare_embeddings(embedding, self.selected_embedding, similarity_strength):
                        bboxes.append(bbox)
                        keypoints.append(landmark)
                        present = True

                        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
                        for i in range(5):
                            cv2.circle(frame, (landmark[i][0], landmark[i][1]), 1, (0, 0, 255), 2)
                presence_mask.append(present)
            else:
                presence_mask.append(False)
            
            writer.write(frame)
        
        self.presence_mask = np.asarray(presence_mask, dtype=bool)
        self.bboxes = np.asarray(bboxes)
        self.kps = np.asarray(keypoints)
        
        video.release()
        writer.release()
        
        return preview_path

    def save_state(self):
        np.save(os.path.join(fc.NPY_FILES_DIR, 'presence_mask.npy'), self.presence_mask)
        np.save(os.path.join(fc.NPY_FILES_DIR, 'bboxes.npy'), self.bboxes)
        np.save(os.path.join(fc.NPY_FILES_DIR, 'keypoints.npy'), self.kps)

def process_video(video_file):
    video_capture = cv2.VideoCapture(video_file.name)
    frames = []
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        frames.append(frame)
    video_capture.release()
    return frames

def update_frame(frames, frame_index):
    return frames[frame_index]

def get_faces_in_frame(face_resolver, frame):
    face_resolver.get_faces(frame)
    faces = face_resolver.faces
    face_images = []
    for face in faces:
        bbox = face.bbox.astype(int)
        face_image = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
        face_images.append(face_image)
    return face_images

def generate_preview(face_resolver, face_index, similarity_strength):
    face_resolver.get_embedding(face_index, face_resolver.faces)
    preview_path = face_resolver.get_preview(similarity_strength)
    return preview_path

def save_data(face_resolver):
    face_resolver.save_state()
    return "Data saved successfully!"

def main():
    with gr.Blocks() as demo:
        video_file = gr.File(label="Upload Video")
        frames = gr.State([])
        frame_index = gr.Slider(minimum=0, maximum=1, step=1, label="Frame Index", interactive=True)
        frame_output = gr.Image(label="Frame Preview")
        face_gallery = gr.Gallery(label="Faces in Frame")
        face_index = gr.Number(label="Selected Face Index", value=0)
        similarity_strength = gr.Slider(minimum=0.1, maximum=2.0, step=0.1, value=1.2, label="Similarity Strength")
        preview_video = gr.Video(label="Preview Video")
        preview_button = gr.Button("Generate Preview")
        save_button = gr.Button("Save Data")
        save_output = gr.Textbox(label="Save Status")

        face_resolver = gr.State(None)

        def load_video(video_file):
            frames_list = process_video(video_file)
            frame_index.update(maximum=len(frames_list) - 1)
            return frames_list, FaceResolver(video_file.name)

        video_file.change(load_video, inputs=[video_file], outputs=[frames, face_resolver])
        frame_index.change(update_frame, inputs=[frames, frame_index], outputs=[frame_output])
        frame_output.change(get_faces_in_frame, inputs=[face_resolver, frame_output], outputs=[face_gallery])
        face_gallery.select(lambda x: x, inputs=[face_index], outputs=[face_index])
        preview_button.click(generate_preview, inputs=[face_resolver, face_index, similarity_strength], outputs=[preview_video])
        save_button.click(save_data, inputs=[face_resolver], outputs=[save_output])

    demo.launch()

if __name__ == "__main__":
    main()

In [ ]:
import gradio as gr
import cv2
import numpy as np
from insightface.app import FaceAnalysis
import os

from sympy import preview

# Assuming helpers.file_check as fc is correctly configured
import helpers.file_check as fc

class FaceResolver:
    def __init__(self, video_path):
        self.video_path = video_path
        self.face_detector = FaceAnalysis(providers=['CUDAExecutionProvider'])
        self.face_detector.prepare(ctx_id=0, det_size=(640, 640))
        
        self.selected_embedding = 0
        self.faces = []

        self.presence_mask = []
        self.bboxes = []
        self.kps = []

    def get_faces(self, frame):
        faces = self.face_detector.get(frame)
        self.faces = faces
    
    def get_embedding(self, index, faces):
        self.selected_embedding = faces[index].normed_embedding

    def _compare_embeddings(self, emb1, emb2, strength = 1.2):
        return np.linalg.norm(emb1 - emb2) < strength

    def get_preview(self, similarity_strength):
        bboxes = []
        presence_mask = []
        keypoints = []

        video = cv2.VideoCapture(self.video_path)

        # Metadata
        fps = video.get(cv2.CAP_PROP_FPS)
        frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Writer Object for output video
        preview_path = os.path.join(fc.MEDIA_DIR, 'preview.mp4')
        writer = cv2.VideoWriter(preview_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

        # Loop through the video
        while True:
            ret, frame = video.read()
            if not ret:
                break
            faces = self.face_detector.get(frame)
            if faces:
                present = False
                for face in faces:
                    bbox = face.bbox.astype(int)
                    landmark = face.kps.astype(int)
                    embedding = face.normed_embedding
                    if self._compare_embeddings(embedding, self.selected_embedding, similarity_strength):
                        bboxes.append(bbox)
                        keypoints.append(landmark)
                        present = True

                        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
                        for i in range(5):
                            cv2.circle(frame, (landmark[i][0], landmark[i][1]), 1, (0, 0, 255), 2)
                presence_mask.append(present)
            else:
                presence_mask.append(False)
            
            writer.write(frame)
        
        self.presence_mask = np.asarray(presence_mask, dtype=bool)
        self.bboxes = np.asarray(bboxes)
        self.kps = np.asarray(keypoints)
        
        video.release()
        writer.release()
        
        return preview_path

    def save_state(self):
        np.save(os.path.join(fc.NPY_FILES_DIR, 'presence_mask.npy'), self.presence_mask)
        np.save(os.path.join(fc.NPY_FILES_DIR, 'bboxes.npy'), self.bboxes)
        np.save(os.path.join(fc.NPY_FILES_DIR, 'keypoints.npy'), self.kps)
        
def process_video(video_file):
    video_capture = cv2.VideoCapture(video_file.name)
    frames = []
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        frames.append(frame)
    video_capture.release()
    return frames

def update_frame(frames, frame_index):
    if frame_index < len(frames):
        return frames[frame_index]
    return None

def get_faces_in_frame(face_resolver, frame):
    face_resolver.get_faces(frame)
    faces = face_resolver.faces
    face_images = []
    for face in faces:
        bbox = face.bbox.astype(int)
        face_image = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
        face_images.append(face_image)
    return face_images

def generate_preview(face_resolver, face_index: gr.SelectData, similarity_strength):
    face_resolver.get_embedding(face_index.index, face_resolver.faces)
    preview_path = face_resolver.get_preview(similarity_strength)
    return preview_path

def save_data(face_resolver):
    face_resolver.save_state()
    return "Data saved successfully!"

def main():
    fc.perform_check()
    with gr.Blocks() as demo:
        video_file = gr.File(label="Upload Video")
        frames = gr.State([])
        frame_index = gr.Slider(minimum=0, maximum=1, step=1, label="Frame Index", interactive=True)
        frame_output = gr.Image(label="Frame Preview")
        face_gallery = gr.Gallery(label="Faces in Frame")
        face_index = gr.Number(label="Selected Face Index", value=0)
        similarity_strength = gr.Slider(minimum=0.1, maximum=2.0, step=0.1, value=1.2, label="Similarity Strength")
        preview_video = gr.Video(label="Preview Video")
        preview_button = gr.Button("Generate Preview")
        save_button = gr.Button("Save Data")
        save_output = gr.Textbox(label="Save Status")

        face_resolver = gr.State(None)

        def load_video(video_file):
            frames_list = process_video(video_file)
            frame_index_update = gr.update(maximum=len(frames_list) - 1) if frames_list else gr.update(maximum=0)
            return frames_list, FaceResolver(video_file.name), frame_index_update

        video_file.change(load_video, inputs=[video_file], outputs=[frames, face_resolver, frame_index])
        frame_index.change(update_frame, inputs=[frames, frame_index], outputs=[frame_output])
        frame_output.change(get_faces_in_frame, inputs=[face_resolver, frame_output], outputs=[face_gallery])
        face_gallery.select(lambda x: x, inputs=[face_index], outputs=[face_index])
        preview_button.click(generate_preview, inputs=[face_resolver, face_index, similarity_strength], outputs=[preview_video])
        save_button.click(save_data, inputs=[face_resolver], outputs=[save_output])

    demo.launch()

if __name__ == "__main__":
    main()

In [ ]:
import gradio as gr

# Sample image URLs
images = [
    "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/3Stars-Micronation_Flag.svg/1200px-3Stars-Micronation_Flag.svg.png",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Circle_-_black_simple.svg/1200px-Circle_-_black_simple.svg.png",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/2/23/Solid_white.svg/2048px-Solid_white.svg.png"
]

def select_image(image, evt: gr.SelectData):
    index = evt.index
    return f"Selected image index: {index}", image

with gr.Blocks() as demo:
    gallery = gr.Gallery(images, label="Select an image")
    output = gr.Textbox(label="Image Index")
    out_img = gr.Textbox(label="Selected Image")

    gallery.select(select_image, gallery, [output,out_img])

demo.launch()

